<a href="https://colab.research.google.com/github/SL-LAIDLAW/projet-outilBioinfo/blob/dev/Projet_Outils_Bioinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet de HMSN204
_Sean LAIDLAW_, _Mathieu Blaison_

### TODO: diagramme de Gantt qui fera une synth`ese des tˆaches effectu´ees et de l’implication de chaque ´etudiant.

Liste des taches et dates (pour faire Gantt après)


*  creation du document colab &  planification du projet = mercredi 27 mars
*  requete NCBI pour sequences sex1 = Samedi, 06 avril  2019 + lundi 08/apr



## Traitements Biopython de SEX1

### Obtention des sequences sauvages et mutés, du NCBI

In [0]:
!pip install biopython
!pip install pandas
from Bio import SeqIO 
from Bio import Entrez

In [0]:
Entrez.email = "sean.laidlaw@etu.umontpellier.fr"
search_string = "SEX1[Gene] AND Arabidopsis thaliana[Organism]"
req = Entrez.esearch(db= "nucleotide", term=search_string)  # ID of fasta of SEX1 gene
res = Entrez.read(req)
res

DictElement({'Count': '5', 'RetMax': '5', 'RetStart': '0', 'IdList': ['1063682343', '1063682341', '334182442', '240254421', '332189094'], 'TranslationSet': [DictElement({'From': 'Arabidopsis thaliana[Organism]', 'To': '"Arabidopsis thaliana"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': 'SEX1[Gene]', 'Field': 'Gene', 'Count': '8', 'Explode': 'N'}, attributes={}), DictElement({'Term': '"Arabidopsis thaliana"[Organism]', 'Field': 'Organism', 'Count': '2696552', 'Explode': 'Y'}, attributes={}), 'AND'], 'QueryTranslation': 'SEX1[Gene] AND "Arabidopsis thaliana"[Organism]'}, attributes={})

In [0]:
sex1_id = res["IdList"]
req.close()
seqgb  = Entrez.efetch (db="nucleotide",id=sex1_id, rettype='fasta')
seqselect = SeqIO.parse(seqgb, "fasta")

sex1_variants = []
for sequence in seqselect:
    sex1_variants.append(sequence.seq)
sex1_variants

[Seq('CACAGATACCGTCCATCAACGTATCACTGGGCTCTCTTTCTCCGTGCCGTTCCT...ATT', SingleLetterAlphabet()),
 Seq('TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATA...AAG', SingleLetterAlphabet()),
 Seq('TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATA...TGG', SingleLetterAlphabet()),
 Seq('CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCC...GGG', SingleLetterAlphabet()),
 Seq('CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCC...GGG', SingleLetterAlphabet())]

## Alignement des séquences associées aux gènes sauvages/mutés

### un alignement par paire global des séquences d’ARNm

Pour aligner nos sequences sauvages avec les mutants nous avons ecrit la fonction AlignSeqs qui effectue un alignement globale en utilisant l'algorithme Needleman–Wunsch.

In [0]:
class compScore:
	def __init__(self,score,direction):
		self.score = str(score)
		self.direction = direction

def AlignSeqs(seqX,seqY):
	seqX = list(" " + seqX)
	seqY = list(" " + seqY)

	match_score = 1
	mismatch_score = -1
	indel_score = -1

	# intitialise dataframe holding scores
	df_dir = pd.DataFrame(index=seqY, columns=seqX)
	df = pd.DataFrame(index=seqY, columns=seqX)
	col_names = list(df.keys())
	row_names = list(df.index)

	# initiate table
	df.iloc[0,0] = 0
	for i in range(len(seqY)):
		df.iloc[i,0] = i * -1  # set row 0 to be -1,-2,-3
	for i in range(len(seqX)):
		df.iloc[0,i] = i * -1  # set col 0 to be -1,-2,-3


	# populate table
	for y in range(1,len(seqY)):
		for x in range(1,len(seqX)):
			current_score = -1000

			# correct offset caused by range starting at 1
			if str(seqX[x]) == str(seqY[y]):
				current_score = match_score
			else:
				current_score = mismatch_score

			df.iloc[y,x] = current_score

			top_left_score = compScore((df.iloc[y-1,x-1]), '\x5C')
			top_score = compScore((df.iloc[y-1,x]), '\x7C')
			left_score = compScore((df.iloc[y,x-1]), '\x3C')

			best_score = top_score.score
			best_direction = top_score.direction
			score_list = [top_left_score, top_score, left_score]
			for score_item in score_list:
				if int(score_item.score) > int(best_score):
					best_direction = score_item.direction
					best_score = score_item.score.strip('\\')

				elif int(score_item.score) == int(best_score):
					if not best_direction.__contains__(score_item.direction):
						best_direction += score_item.direction

			best_score = int(best_score)
			best_score += current_score

			df.iloc[y,x] = best_score
			df_dir.iloc[y,x] = best_direction

	# print(df)
	# print(df_dir)

	# Start traceback
	my_alignment_x =[]
	my_alignment_y =[]
	bottom_right_y = list(df.shape)[0] -1
	bottom_right_x = list(df.shape)[1] -1

	x = bottom_right_x
	y = x - (bottom_right_x - bottom_right_y)
	while x != 0:

		current_coordinates = [y,x]
		top_coordinates = [y-1,x]
		left_coordinates = [y,x-1]
		top_left_coordinates = [y-1,x-1]

		current_nucleotide_x = col_names[x]
		current_nucleotide_y = row_names[y]
		current_direction = df_dir.iloc[y,x]

		if current_direction.__contains__('\\'):
			# move coordinates to top_left
			x = top_left_coordinates[1]
			y = top_left_coordinates[0]
			my_alignment_x.append(current_nucleotide_x)
			my_alignment_y.append(current_nucleotide_y)

		elif current_direction.__contains__('|'):
			# move coordinates to top
			x = top_coordinates[1]
			y = top_coordinates[0]
			my_alignment_x.append('-')
			my_alignment_y.append(current_nucleotide_y)

		elif current_direction.__contains__('<'):
			# move coordinates to left
			x = left_coordinates[1]
			y = left_coordinates[0]
			my_alignment_x.append(current_nucleotide_x)
			my_alignment_y.append('-')

	my_alignment_x.reverse()
	my_alignment_y.reverse()
	my_alignment_x = str(my_alignment_x).strip('[').strip(']').replace(',','').replace(' ','')
	my_alignment_y = str(my_alignment_y).strip('[').strip(']').replace(',','').replace(' ','')
	return my_alignment_x, my_alignment_y


### donner les positions des variations sauvage/muté

In [0]:
myseq = myseqs[0]
myseq

Seq('CACAGATACCGTCCATCAACGTATCACTGGGCTCTCTTTCTCCGTGCCGTTCCT...ATT', SingleLetterAlphabet())

### taux de GC / traduction pour obtenir les séquences protéiques

récupation séquence (à modifier)



In [0]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
# replace this with Entrez
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", IUPAC.unambiguous_dna)

In [0]:
from Bio.SeqUtils import GC
GC(myseq)

43.37823834196891

In [0]:
coding_dna.translate()

/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:2609: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


Seq('HRYRPSTYHWALFLRAVPPTCSNNDVPRGGIEG*SVVFYFDYT*VYVCVSE*RR...NSN', HasStopCodon(ExtendedIUPACProtein(), '*'))

# Base de données

## Reflection sur le type de base de données

On a eu le choix d’un des SGBD vu ce semestre : chado/postgresql (module phenotype), relationnel seul (oracle, postgresqlou mysql), neo4j, couchdb, triplestore jena/rdf. 

On a choisi d'eviter les base de données NoSQL car nous avons plus d'experience avec ceux là. 

Notre choix s'est ainsi  porté sur la base de données Chado. Celle-ci est conçue spécialement pour gérer les représentations complexes de données biologiques.

## Modelisation de la base de données

## Implantation de la base de données et peuplement

In [0]:
from Bio import SeqIO
import pandas as pd

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/SL-LAIDLAW/projet-outilBioinfo/master/Data_Projet_Entete.csv",sep=";")
df['Mutant'] = False
df['Mutant'].loc[df['Genotype'] != "Col_0"] = True
df.loc[df['Mutant'] == True]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Genotype,Milieu,Plate,Id,PR length,Tortuosity,RootArea,LR length,Grouping,Plants/group,distanceFromHypocotyl std (cm),ShootArea,Chlorophyll (a.u),Mutant
186,nrt,Milieu_1,Plate_1,F1_1,11.163,1.102,0.259,10.14,1,7,2.348,0.187,4.250,True
187,nrt,Milieu_1,Plate_1,F1_2,7.419,1.082,0.183,5.00,2,2,1.473,0.132,3.033,True
188,nrt,Milieu_1,Plate_1,F1_3,7.537,1.116,0.183,5.00,2,2,1.473,0.041,0.674,True
189,nrt,Milieu_1,Plate_1,F1_4,7.380,1.087,0.259,10.14,1,7,2.348,0.171,3.394,True
190,nrt,Milieu_1,Plate_1,F1_5,7.545,1.085,0.259,10.14,1,7,2.348,0.049,0.958,True
191,nrt,Milieu_1,Plate_1,F1_6,10.897,1.114,0.259,10.14,1,7,2.348,0.099,2.176,True
192,nrt,Milieu_1,Plate_1,F1_7,7.259,1.085,0.259,10.14,1,7,2.348,0.150,2.597,True
193,nrt,Milieu_1,Plate_1,F1_8,6.605,1.092,0.259,10.14,1,7,2.348,0.017,0.211,True
194,nrt,Milieu_1,Plate_1,F1_9,4.745,1.106,0.259,10.14,1,7,2.348,0.132,2.527,True
195,nrt,Milieu_1,Plate_1,F1_10,6.390,1.088,0.189,7.75,3,4,2.274,0.022,0.302,True


In [0]:
cov = df.cov()
cov

,PR length,Tortuosity,RootArea,LR length,Grouping,Plants/group,distanceFromHypocotyl std (cm),ShootArea,Chlorophyll (a.u)
PR length,10.748196,0.028717,0.360917,14.306058,-5.536155,5.666499,2.713649,0.177008,3.564540
Tortuosity,0.028717,0.005974,0.001301,0.050465,-0.052913,0.022540,0.010562,0.000559,0.012890
RootArea,0.360917,0.001301,0.021034,0.777904,-0.281867,0.333221,0.134795,0.008780,0.173087
LR length,14.306058,0.050465,0.777904,34.435065,-9.316980,11.438061,5.365691,0.336841,6.175700
Grouping,-5.536155,-0.052913,-0.281867,-9.316980,17.675330,-6.068031,-1.994934,-0.133922,-2.935009
Plants/group,5.666499,0.022540,0.333221,11.438061,-6.068031,10.899694,2.463164,0.163863,3.474913
distanceFromHypocotyl std (cm),2.713649,0.010562,0.134795,5.365691,-1.994934,2.463164,1.161365,0.058837,1.158878
ShootArea,0.177008,0.000559,0.008780,0.336841,-0.133922,0.163863,0.058837,0.009249,0.179739
Chlorophyll (a.u),3.564540,0.012890,0.173087,6.175700,-2.935009,3.474913,1.158878,0.179739,3.828083
